In [ ]:
import sys
import os# Navigate up one level to the parent directory and append it to sys.path
import importlib
import requests

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import nfl_data_py as nfl
import pandas as pd
from src import utils
from src import homers
import plotly.express as px
from src.config import LOGO_PATH,  BETTING_PATH

POWER_RATING_PATH = BETTING_PATH / 'power-ratings.pkl'

In [ ]:
SEASON, WEEK = 2023, 11
HFA = 1.5
AVG_POINTS = 21.5

In [ ]:
def read_power(season, week):
    power_df = pd.read_clipboard()
    assert len(power_df) == 32
    power_df['season']=season
    power_df['week']=week
    power_df['power_id'] = power_df.apply(lambda row: f'{row.season}_{row.week}_{row.team}', axis=1)
    power_df = power_df.set_index('power_id').sort_values(by=['ovr', 'off'], ascending=False)
    return power_df

In [ ]:
current_week = read_power(SEASON, WEEK)
pwr = pd.read_pickle(POWER_RATING_PATH)
pwr = pd.concat([pwr, current_week])
pwr.to_pickle(POWER_RATING_PATH)

In [ ]:
sched = nfl.import_schedules([SEASON])

In [ ]:
sched = sched[sched['week']==WEEK][['game_id', 'away_team', 'home_team']].copy().set_index('game_id')
sched.head()

In [ ]:
predict_home_score = lambda row: AVG_POINTS + row.home_off - row.away_def + HFA / 2
predict_away_score = lambda row: AVG_POINTS + row.away_off - row.home_def - HFA / 2

# Evaluate Movement between Weeks

In [ ]:

previous_power = pd.read_clipboard()
previous_power

In [ ]:
movement = pd.merge(power_df, previous_power, on='team', suffixes=('_cur', '_prev'))
for col in ['ovr', 'off', 'def']:
    movement[f'{col}_dif'] = movement[f'{col}_cur'] - movement[f'{col}_prev']
movement

In [ ]:
# utils.download_team_pngs()

In [ ]:
col = 'ovr_cur'

# Create the bar chart
fig = px.bar(movement, x=col, y='team', orientation='h', color='team', color_discrete_map=utils.team_unique_colors)
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(
    height=1000,
    width=800,
    xaxis_title=f'Generic Points Favored',
    yaxis_title='Teams',
    title=f'Week {WEEK} Power Ratings',
)
fig.update_traces(showlegend=False)

# Iterate through the data and add logos to the chart
for index, row in movement.iterrows():
    team = row['team']
    scale = 1.25
    fig.add_layout_image(
        dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
            x=row[col],  # Adjust the position
            y=team,
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",        
        )
    )

# Show the chart
fig.show()

In [ ]:
col = 'ovr_dif'

# Create the bar chart
fig = px.bar(movement, x=col, y='team', orientation='h', color='team', color_discrete_map=utils.team_unique_colors)
fig.update_yaxes(categoryorder='total ascending')
fig.update_layout(
    height=1000,
    width=800,
    xaxis_title=f'Generic Points Favored',
    yaxis_title='Teams',
    title=f'Week {WEEK} Power Ratings',
)
fig.update_traces(showlegend=False)

# Iterate through the data and add logos to the chart
for index, row in movement.iterrows():
    team = row['team']
    scale = 1.25
    fig.add_layout_image(
        dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
            x=row[col],  # Adjust the position
            y=team,
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",        
        )
    )

# Show the chart
fig.show()

In [ ]:


fig = px.scatter(power_df, x='off', y='def', opacity=0, color='team', color_discrete_map=utils.team_unique_colors)
fig.update_layout(
    height=600,
    width=1000,

    xaxis_title=f'Generic Points Favored',
    yaxis_title='Teams',
    title=f'Week {WEEK} Power Ratings',
)
fig.update_traces(showlegend=False)
# Iterate through the data and add logos to the chart
for index, row in power_df.iterrows():
    team = row['team']
    scale = 0.9
    fig.add_layout_image(
        dict(source=f'https://a.espncdn.com/i/teamlogos/nfl/500/{team}.png',
            x=row['off'],  # Adjust the position
            y=row['def'],
            xref="x",
            yref="y",
            sizex=scale,  # Adjust the size
            sizey=scale,  # Adjust the size
            sizing="contain",
            opacity=1,
            xanchor="center",
            yanchor="middle",        
        )
    )

fig.show()